## MAP - Charting Student Math Misunderstandings

---

### Dataset (train/test.csv)
- QuestionId: Unique question identifier
- QuestionText: The text of the question
- MC_Answer: The multiple-choice answer the student selected
- Student Explanation: A student's explanation for choosing a MC_Answer
- Category(train only): A classification of relationship between MC_Answer and Student Explanation
- Misconception(train only): Only applicable when Category contains a misconception, otherwise is NA

### Submission (sample_submission.csv)
- Category:Misconception: The predicted classification Category concatenated with Misconception by a colon

### 1. Data upload

In [1]:
import sys
from pathlib import Path

ROOT_DIR = Path().resolve().parent
sys.path.append(str(ROOT_DIR))

In [2]:
from src.data_load import load_train, load_test

train = load_train()
test = load_test()

print(train.head())
print(test.head())

   row_id  QuestionId                                       QuestionText  \
0       0       31772  What fraction of the shape is not shaded? Give...   
1       1       31772  What fraction of the shape is not shaded? Give...   
2       2       31772  What fraction of the shape is not shaded? Give...   
3       3       31772  What fraction of the shape is not shaded? Give...   
4       4       31772  What fraction of the shape is not shaded? Give...   

           MC_Answer                                 StudentExplanation  \
0  \( \frac{1}{3} \)                  0ne third is equal to tree nineth   
1  \( \frac{1}{3} \)  1 / 3 because 6 over 9 is 2 thirds and 1 third...   
2  \( \frac{1}{3} \)  1 3rd is half of 3 6th, so it is simplee to un...   
3  \( \frac{1}{3} \)        1 goes into everything and 3 goes into nine   
4  \( \frac{1}{3} \)                    1 out of every 3 isn't coloured   

       Category Misconception  
0  True_Correct           NaN  
1  True_Correct           Na

In [3]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36696 entries, 0 to 36695
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              36696 non-null  int64 
 1   QuestionId          36696 non-null  int64 
 2   QuestionText        36696 non-null  object
 3   MC_Answer           36696 non-null  object
 4   StudentExplanation  36696 non-null  object
 5   Category            36696 non-null  object
 6   Misconception       9860 non-null   object
dtypes: int64(2), object(5)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              3 non-null      int64 
 1   QuestionId          3 non-null      int64 
 2   QuestionText        3 non-null      object
 3   MC_Answer           3 non-null      object
 4   StudentExpla

In [5]:
print(train.describe())
print(test.describe())

             row_id     QuestionId
count  36696.000000   36696.000000
mean   18347.500000   46356.825104
std    10593.367076   25617.723392
min        0.000000   31772.000000
25%     9173.750000   31777.000000
50%    18347.500000   32833.000000
75%    27521.250000   33474.000000
max    36695.000000  109465.000000
        row_id    QuestionId
count      3.0      3.000000
mean   36697.0  32126.333333
std        1.0    613.723336
min    36696.0  31772.000000
25%    36696.5  31772.000000
50%    36697.0  31772.000000
75%    36697.5  32303.500000
max    36698.0  32835.000000
